In [1]:
from src import SharePricePrediction, PredictValues

In [ ]:
def getFundamentals(c_name, future_years):
    predicted_values = PredictValues(c_name=c_name)
    fundamental_values = predicted_values.getFutureValues(future_year=future_years)
    print(fundamental_values)
    return (fundamental_values)

In [ ]:
fundametal_values = getFundamentals('ITC', 5)

In [ ]:
for val in fundametal_values:
    print(f'{val} : {fundametal_values[val]}')

In [ ]:
test_rev = fundametal_values['future_revenue']

test_rev_arr = [round(float(val), 2) for val in test_rev]
print(test_rev_arr)

In [2]:
import json  # Import JSON module to handle JSON strings

def getSharePrice(c_name, future_years):
    share_price_prediction = SharePricePrediction(company_name=c_name)
    share_price = share_price_prediction.SharePrice(future_years=future_years)
    previous_share_price = share_price['previous_share_price']
    future_share_price = share_price['future_share_price']
    
    # Parse JSON strings to Python dictionaries
    data = {
        'previous_share_price': json.loads(previous_share_price.to_json(orient='records', date_format='iso')),
        'future_share_price': json.loads(future_share_price.to_json(orient='records', date_format='iso'))
    }
    return data

In [3]:
share_price_data = getSharePrice('ITC', 5)

77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117

In [4]:
val = share_price_data['previous_share_price']

In [6]:
print(val[0])

{'Date': '2015-04-10T00:00:00.000', 'Close': 146.8356323242, 'ema200': 146.8356323242}
